# Operations on Word Vectors
Welcome to your first assignment of Week 2, Course 5 of the Deep Learning Specialization!

Because word embeddings are very computationally expensive to train, most ML practitioners will load a pre-trained set of embeddings. In this notebook you'll try your hand at loading, measuring similarity between, and modifying pre-trained embeddings.

## After this assignment you'll be able to:

- Explain how word embeddings capture relationships between words
- Load pre-trained word vectors
- Measure similarity between word vectors using cosine similarity
- Use word embeddings to solve word analogy problems such as Man is to Woman as King is to ______.
  
At the end of this notebook you'll have a chance to try an optional exercise, where you'll modify word embeddings to reduce their gender bias. Reducing bias is an important consideration in ML, so you're encouraged to take this challenge!

In [1]:
import numpy as np
from w2v_utils import *

In [24]:
words, word_to_vec_map = read_glove_vecs('data/glove.6B.50d.txt')

## 2 - Embedding Vectors Versus One-Hot Vectors
Recall from the lesson videos that one-hot vectors don't do a good job of capturing the level of similarity between words. This is because every one-hot vector has the same Euclidean distance from any other one-hot vector.

Embedding vectors, such as GloVe vectors, provide much more useful information about the meaning of individual words.
Now, see how you can use GloVe vectors to measure the similarity between two words!

## 3 - Cosine Similarity
To measure the similarity between two words, you need a way to measure the degree of similarity between two embedding vectors for the two words. Given two vectors __u__ and __v__, cosine similarity is defined as follows:
$$CosineSimilarity(u,v) = \frac{u.v}{||u||_2 \;||v||_2} = cos(θ) \tag {1}$$
 
 

 - u.v is the dot product (or inner product) of two vectors
 - $||u||_2$is the norm (or length) of the vector 
 - θ is the angle between u and v
- The cosine similarity depends on the angle between u and v
	- If u and v are very similar, their cosine similarity will be close to 1.
	- If they are dissimilar, the cosine similarity will take a smaller value.

![Cosine](images/cosine.png)

## Exercise 1 - cosine_similarity
Implement the function cosine_similarity() to evaluate the similarity between word vectors.

Reminder: The norm of u is defined as $||u||_2$ = $\sqrt{∑^n_{i=1} u^2_i}$

$$CosineSimilarity(u,v) = \frac{u.v}{||u||_2 \;||v||_2} = cos(θ)$$

### Additional Hints
You may find np.dot, np.sum, or np.sqrt useful depending upon the implementation that you choose.

In [43]:
def cosine_similarity(u,v):
    if np.all(u==v):
        return 1
    dot = np.dot(u,v)
    norm_u = np.sqrt(np.sum(u*u))
    norm_v = np.sqrt(np.sum(v*v))

    cosine_similarity = dot / (norm_u * norm_v)
    return cosine_similarity

In [26]:
father = word_to_vec_map["father"]
mother = word_to_vec_map["mother"]
ball = word_to_vec_map["ball"]
crocodile = word_to_vec_map["crocodile"]
france = word_to_vec_map["france"]
italy = word_to_vec_map["italy"]
paris = word_to_vec_map["paris"]
rome = word_to_vec_map["rome"]

print("cosine_similarity(father, mother) = ", cosine_similarity(father, mother))
print("cosine_similarity(ball, crocodile) = ",cosine_similarity(ball, crocodile))
print("cosine_similarity(france - paris, rome - italy) = ",cosine_similarity(france - paris, rome - italy))
# END SKIP FOR GRADING

# PUBLIC TESTS
def cosine_similarity_test(target):
    a = np.random.uniform(-10, 10, 10)
    b = np.random.uniform(-10, 10, 10)
    c = np.random.uniform(-1, 1, 23)
        
    assert np.isclose(cosine_similarity(a, a), 1), "cosine_similarity(a, a) must be 1"
    assert np.isclose(cosine_similarity((c >= 0) * 1, (c < 0) * 1), 0), "cosine_similarity(a, not(a)) must be 0"
    assert np.isclose(cosine_similarity(a, -a), -1), "cosine_similarity(a, -a) must be -1"
    assert np.isclose(cosine_similarity(a, b), cosine_similarity(a * 2, b * 4)), "cosine_similarity must be scale-independent. You must divide by the product of the norms of each input"

    print("\033[92mAll test passed!")
    
cosine_similarity_test(cosine_similarity)

cosine_similarity(father, mother) =  0.8909038442893615
cosine_similarity(ball, crocodile) =  0.27439246261379424
cosine_similarity(france - paris, rome - italy) =  -0.6751479308174201
All test passed!


## 4 - Word Analogy Task
- In the word analogy task, complete this sentence:
"a is to b as c is to ____".

- An example is:
'man is to woman as king is to queen' .

- You're trying to find a word d, such that the associated word vectors $e_a, e_b, e_c, e_d$
 are related in the following manner:
$e_b - e_a ≈ e_d - e_c$
- Measure the similarity between $e_b - e_a$ and $e_d - e_c$ using cosine similarity.


## Exercise 2 - complete_analogy

In [30]:
def complete_analogy(word_a, word_b, word_c, word_to_vec_map):
    """
    Performs the word analogy task as explained above: a is to b as c is to ____. 
    
    Arguments:
    word_a -- a word, string
    word_b -- a word, string
    word_c -- a word, string
    word_to_vec_map -- dictionary that maps words to their corresponding vectors. 
    
    Returns:
    best_word --  the word such that v_b - v_a is close to v_best_word - v_c, as measured by cosine similarity
    """
    
    # convert words to lowercase
    word_a, word_b, word_c = word_a.lower(), word_b.lower(), word_c.lower()
    
    ### START CODE HERE ###
    # Get the word embeddings e_a, e_b and e_c (≈1-3 lines)
    e_a, e_b, e_c = word_to_vec_map[word_a], word_to_vec_map[word_b], word_to_vec_map[word_c]
    ### END CODE HERE ###
    
    words = word_to_vec_map.keys()
    max_cosine_sim = -100              # Initialize max_cosine_sim to a large negative number
    best_word = None                   # Initialize best_word with None, it will help keep track of the word to output
    
    # loop over the whole word vector set
    for w in words:   
        # to avoid best_word being one the input words, skip the input word_c
        # skip word_c from query
        if w == word_c:
            continue
        
        ### START CODE HERE ###
        # Compute cosine similarity between the vector (e_b - e_a) and the vector ((w's vector representation) - e_c)  (≈1 line)
        cosine_sim = cosine_similarity(e_b - e_a, word_to_vec_map[w] - e_c)
        
        # If the cosine_sim is more than the max_cosine_sim seen so far,
            # then: set the new max_cosine_sim to the current cosine_sim and the best_word to the current word (≈3 lines)
        if cosine_sim > max_cosine_sim:
            max_cosine_sim = cosine_sim
            best_word = w
        ### END CODE HERE ###
        
    return best_word

In [31]:
# PUBLIC TEST
def complete_analogy_test(target):
    a = [3, 3] # Center at a
    a_nw = [2, 4] # North-West oriented vector from a
    a_s = [3, 2] # South oriented vector from a
    
    c = [-2, 1] # Center at c
    # Create a controlled word to vec map
    word_to_vec_map = {'a': a,
                       'synonym_of_a': a,
                       'a_nw': a_nw, 
                       'a_s': a_s, 
                       'c': c, 
                       'c_n': [-2, 2], # N
                       'c_ne': [-1, 2], # NE
                       'c_e': [-1, 1], # E
                       'c_se': [-1, 0], # SE
                       'c_s': [-2, 0], # S
                       'c_sw': [-3, 0], # SW
                       'c_w': [-3, 1], # W
                       'c_nw': [-3, 2] # NW
                      }
    
    # Convert lists to np.arrays
    for key in word_to_vec_map.keys():
        word_to_vec_map[key] = np.array(word_to_vec_map[key])
            
    assert(target('a', 'a_nw', 'c', word_to_vec_map) == 'c_nw')
    assert(target('a', 'a_s', 'c', word_to_vec_map) == 'c_s')
    assert(target('a', 'synonym_of_a', 'c', word_to_vec_map) != 'c'), "Best word cannot be input query"
    assert(target('a', 'c', 'a', word_to_vec_map) == 'c')

    print("\033[92mAll tests passed")
    
complete_analogy_test(complete_analogy)

All tests passed


C:\Users\theha\AppData\Local\Temp\ipykernel_13984\4163701915.py:8: RuntimeWarning: invalid value encountered in true_divide
  cosine_similarity = dot / (norm_u * norm_v)


In [32]:
# START SKIP FOR GRADING
triads_to_try = [('italy', 'italian', 'spain'), ('india', 'delhi', 'japan'), ('man', 'woman', 'boy'), ('small', 'smaller', 'large')]
for triad in triads_to_try:
    print ('{} -> {} :: {} -> {}'.format( *triad, complete_analogy(*triad, word_to_vec_map)))

italy -> italian :: spain -> spanish
india -> delhi :: japan -> tokyo
man -> woman :: boy -> girl
small -> smaller :: large -> smaller


# 5 - Debiasing Word Vectors (OPTIONAL/UNGRADED)
In the following exercise, you'll examine gender biases that can be reflected in a word embedding, and explore algorithms for reducing the bias. In addition to learning about the topic of debiasing, this exercise will also help hone your intuition about what word vectors are doing. This section involves a bit of linear algebra, though you can certainly complete it without being an expert! Go ahead and give it a shot. This portion of the notebook is optional and is not graded...so just have fun and explore.

First, see how the GloVe word embeddings relate to gender. You'll begin by computing a vector g = $e_{woman} - e_{man}$, where $e_{woman}$ represents the word vector corresponding to the word woman, and $e_man$ corresponds to the word vector corresponding to the word man. The resulting vector 
__g__ roughly encodes the concept of "gender".

You might get a more accurate representation if you compute $g1 = e_{mother} - e_{father}, = e_{girl} - e_{boy},$ etc. and average over them, but just using $e_{woman} -e_{man}$ will give good enough results for now.

In [33]:
g = word_to_vec_map['woman'] - word_to_vec_map['man']
print(g)

[-0.087144    0.2182     -0.40986    -0.03922    -0.1032      0.94165
 -0.06042     0.32988     0.46144    -0.35962     0.31102    -0.86824
  0.96006     0.01073     0.24337     0.08193    -1.02722    -0.21122
  0.695044   -0.00222     0.29106     0.5053     -0.099454    0.40445
  0.30181     0.1355     -0.0606     -0.07131    -0.19245    -0.06115
 -0.3204      0.07165    -0.13337    -0.25068714 -0.14293    -0.224957
 -0.149       0.048882    0.12191    -0.27362    -0.165476   -0.20426
  0.54376    -0.271425   -0.10245    -0.32108     0.2516     -0.33455
 -0.04371     0.01258   ]


Now, consider the cosine similarity of different words with __g__. What does a positive value of similarity mean, versus a negative cosine similarity?

In [34]:
print ('List of names and their similarities with constructed vector:')

# girls and boys name
name_list = ['john', 'marie', 'sophie', 'ronaldo', 'priya', 'rahul', 'danielle', 'reza', 'katy', 'yasmin']

for w in name_list:
    print (w, cosine_similarity(word_to_vec_map[w], g))

List of names and their similarities with constructed vector:
john -0.23163356145973724
marie 0.31559793539607295
sophie 0.31868789859418784
ronaldo -0.3124479685032943
priya 0.17632041839009407
rahul -0.16915471039231722
danielle 0.24393299216283892
reza -0.0793042967219955
katy 0.2831068659572615
yasmin 0.23313857767928753


As you can see, female first names tend to have a positive cosine similarity with our constructed vector 
, while male first names tend to have a negative cosine similarity. This is not surprising, and the result seems acceptable.

Now try with some other words:

In [36]:
print('Other words and their similarities:\n')
word_list = ['lipstick', 'guns', 'science', 'arts', 'literature', 'warrior','doctor', 'tree', 'receptionist', 
             'technology',  'fashion', 'teacher', 'engineer', 'pilot', 'computer', 'singer']
for w in word_list:
    print (w, cosine_similarity(word_to_vec_map[w], g))

Other words and their similarities:

lipstick 0.2769191625638267
guns -0.1888485567898898
science -0.060829065409296994
arts 0.008189312385880351
literature 0.0647250443345993
warrior -0.20920164641125288
doctor 0.1189528941093504
tree -0.07089399175478091
receptionist 0.3307794175059374
technology -0.13193732447554293
fashion 0.03563894625772701
teacher 0.1792092343182567
engineer -0.08039280494524072
pilot 0.0010764498991917152
computer -0.10330358873850494
singer 0.1850051813649629


Do you notice anything surprising? It is astonishing how these results reflect certain unhealthy gender stereotypes. For example, we see “computer” is negative and is closer in value to male first names, while “literature” is positive and is closer to female first names. Ouch!

You'll see below how to reduce the bias of these vectors, using an algorithm due to Boliukbasi et al., 2016. Note that some word pairs such as "actor"/"actress" or "grandmother"/"grandfather" should remain gender-specific, while other words such as "receptionist" or "technology" should be neutralized, i.e. not be gender-related. You'll have to treat these two types of words differently when debiasing.

You'll see below how to reduce the bias of these vectors, using an algorithm due to Boliukbasi et al., 2016. Note that some word pairs such as "actor"/"actress" or "grandmother"/"grandfather" should remain gender-specific, while other words such as "receptionist" or "technology" should be neutralized, i.e. not be gender-related. You'll have to treat these two types of words differently when debiasing.

# 5.1 - Neutralize Bias for Non-Gender Specific Words
The figure below should help you visualize what neutralizing does. If you're using a 50-dimensional word embedding, the 50 dimensional space can be split into two parts: The bias-direction __g__, and the remaining 49 dimensions, which is called __g⊥__ here. In linear algebra, we say that the 49-dimensional __g⊥__ is perpendicular (or "orthogonal") to __g__, meaning it is at 90 degrees to __g__
. The neutralization step takes a vector such as __$e_{receptionist}$__ and zeros out the component in the direction of __g__, giving us $e^{debiased}_{receptionist}$.

Even though __g⊥__ is 49-dimensional, given the limitations of what you can draw on a 2D screen, it's illustrated using a 1-dimensional axis below.

![Bias](images/bias.png)

## Exercise 3 - neutralize
Implement neutralize() to remove the bias of words such as "receptionist" or "scientist."

Given an input embedding __e__, you can use the following formulas to compute e^{debiased} :

$$e^{biasComponent} = \frac {e.g}{||g||^2_2} * g \tag {2}$$

$$e^{debiased} =  e - e^{biasComponent} \tag {3}$$

If you are an expert in linear algebra, you may recognize e^{bias_component} as the projection of __e__ onto the direction __g__. If you're not an expert in linear algebra, don't worry about this. ;)

In [40]:
def neutralize(word, g, word_to_vec_map):
    e = word_to_vec_map[word]
    g_norm_squared = np.sum(g*g)
    e_biascomponent = (np.dot(e,g) / g_norm_squared) * g
    e_debiased = e - e_biascomponent
    return e_debiased

In [41]:
e = "receptionist"
print("cosine similarity between " + e + "and g, before neutralizing: ", cosine_similarity(word_to_vec_map["receptionist"], g))

e_debiased = neutralize("receptionist", g, word_to_vec_map)
print("cosine similarity between " + e + "and g, after neutralizing: ", cosine_similarity(e_debiased, g))


cosine similarity between receptionistand g, before neutralizing:  0.3307794175059374
cosine similarity between receptionistand g, after neutralizing:  -5.2569290990191626e-17


$$ μ = \frac {e_{w1}+e_{w2}}{2} \tag{4}$$
$$ μ_B =\frac {μ \;. bias\_axis}{||bias\_axis||^2_2} * bias\_axis \tag {5}$$

$$ μ⊥ = μ- μ_B \tag {6} $$ 

$$ e_{w1B} = \frac {e_{w1} \;. bias\_axis} { ||bias\_axis||^2_2} * bias\_axis \tag {7}$$
$$ e_{w2B} = \frac {e_{w2} \;. bias\_axis} { ||bias\_axis||^2_2} * bias\_axis \tag {8}$$
$$ e^{corrected}_{w1B} = \sqrt{|1 - \;||μ_⊥||^2_2 \;|} * \frac {e_{w1B} - μB}{||(e_{w1} - μ_⊥) - μ_B||_2} \tag {9}$$
$$ e^{corrected}_{w2B} = \sqrt{|1 - \;||μ_⊥||^2_2 \;|} * \frac {e_{w2B} - μB}{||(e_{w2} - μ_⊥) - μ_B||_2} \tag {10}$$
$$ e_1 = e^{corrected}_{w1B} + μ_⊥ \tag {11}$$
$$ e_2 = e^{corrected}_{w2B} + μ_⊥ \tag {12}$$

## Exercise 4 - equalize
Implement the equalize() function below.

Use the equations above to get the final equalized version of the pair of words. Good luck!

- __Hint__

- Use np.linalg.norm

- Reminder: The norm of u is defined as $||u||_2$ = $\sqrt{∑^n_{i=1} u^2_i}$

- norm_u = np.sqrt(np.sum(u*u))

In [52]:
def equalize(pair, bias_axis, word_to_vec_map):

    #step 1
    w1, w2 = pair
    e_w1, e_w2 = word_to_vec_map[w1], word_to_vec_map[w2]

    #step2
    mu = (e_w1 + e_w2 ) / 2

    #step3
    bias_axis_sq = np.sum(bias_axis * bias_axis)
    mu_B = (np.dot(mu, bias_axis) / bias_axis_sq) * bias_axis
    mu_orth = mu - mu_B

    #step 4
    e_w1B = (np.dot(e_w1, bias_axis) / bias_axis_sq) * bias_axis
    e_w2B = (np.dot(e_w2, bias_axis) / bias_axis_sq) * bias_axis

    #step5
    correction_term = np.sqrt(np.abs(1 - np.sum(mu_orth **2)))
    
    e_w1B_norm = np.sqrt(np.sum(((e_w1 - mu_orth) - mu_B) ** 2))
    e_w2B_norm = np.sqrt(np.sum(((e_w2 - mu_orth) - mu_B) ** 2))
    
    corrected_e_w1B = correction_term * ((e_w1B - mu_B) / e_w1B_norm)
    corrected_e_w2B = correction_term * ((e_w2B - mu_B) / e_w2B_norm)

    #step6
    e1 = corrected_e_w1B + mu_orth
    e2 = corrected_e_w2B + mu_orth

    return e1, e2

In [53]:
print("cosine similarities before equalizing:")
print("cosine_similarity(word_to_vec_map[\"man\"], gender) = ", cosine_similarity(word_to_vec_map["man"], g))
print("cosine_similarity(word_to_vec_map[\"woman\"], gender) = ", cosine_similarity(word_to_vec_map["woman"], g))
print()
e1, e2 = equalize(("man", "woman"), g, word_to_vec_map)
print("cosine similarities after equalizing:")
print("cosine_similarity(e1, gender) = ", cosine_similarity(e1, g))
print("cosine_similarity(e2, gender) = ", cosine_similarity(e2, g))

cosine similarities before equalizing:
cosine_similarity(word_to_vec_map["man"], gender) =  -0.1171109576533683
cosine_similarity(word_to_vec_map["woman"], gender) =  0.3566661884627037

cosine similarities after equalizing:
cosine_similarity(e1, gender) =  -0.7004364289309387
cosine_similarity(e2, gender) =  0.7004364289309387


## 6 - References
- The debiasing algorithm is from Bolukbasi et al., 2016, Man is to Computer Programmer as Woman is to Homemaker? Debiasing Word Embeddings
- The GloVe word embeddings were due to Jeffrey Pennington, Richard Socher, and Christopher D. Manning. (https://nlp.stanford.edu/projects/glove/)